In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt

In [3]:
btc_raw_df = pd.read_parquet("/home/giujorge/Documents/Projets/bot_strategies/data/raw/BTC-USDT.csv.parquet.gzip")
btc_raw_df

,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.77518300,1502942459999,7564.90685084,3,0.07518300,320.39085084,7960.54017996
2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.00000000,1502942519999,0.00000000,0,0.00000000,0.00000000,7960.45262989
2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.26107400,1502942579999,1117.54292144,2,0.26107400,1117.54292144,7960.37529526
2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.01200800,1502942639999,51.17185184,3,0.01200800,51.17185184,7960.37529526
2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.14079600,1502942699999,599.99933808,1,0.14079600,599.99933808,7960.37529526
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-18 14:09:00,39423.05,39423.06,39377.05,39385.62,22.37338000,1650290999999,881512.83100580,758,6.59638000,259885.09290650,0
2022-04-18 14:10:00,39385.61,39401.54,39370.24,39400.00,29.43069000,1650291059999,1159033.25545720,788,14.55146000,573044.31840130,0
2022-04-18 14:11:00,39400.00,39444.80,39399.83,39442.55,25.65637000,1650291119999,1011437.53518520,720,13.72377000,541044.94403500,0


In [9]:
def Intradaytrend(df, entry, exit):
    ret_120min = df.iloc[120].open /df.iloc[0].open - 1
    tickret = df.open.pct_change()
    if ret_120min > entry:
        buyprice = df.iloc[121].open
        buytime = df.iloc[121].name
        cumulated = (tickret.loc[buytime:] + 1).cumprod() - 1
        exittime = cumulated[(cumulated < -exit) | (cumulated > exit)].first_valid_index()
        if exittime == None:
            exitprice = df.iloc[-1].open
        else :
            exitprice = df.loc[exittime + dt.timedelta(minutes=1)].open
        profit = exitprice - buyprice
        profitrel = profit/buyprice
        return profitrel
    else:
        return None


In [12]:
Intradaytrend(btc_raw_df[['open']], 0.01, 0.01)

0.014490234791394534

In [14]:
df1 = btc_raw_df.resample('1D').agg({'open' : 'min'})
df1

,open
timestamp,
2017-08-17,4200.74
2017-08-18,3949.36
2017-08-19,3870.62
2017-08-20,4052.82
2017-08-21,3914.00
...,...
2022-04-14,39587.03
2022-04-15,39773.00
2022-04-16,40023.50
